<a href="https://colab.research.google.com/github/Ddne59/Titanic---Machine-Learning-from-Disaster/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [413]:
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [414]:
with open('/content/drive/MyDrive/train.csv') as f:
  df = pd.read_csv(f)
  print("Head : \n", df.head() , "\n")
  print("Shape \n",df.shape, "\n")
  print("Info \n ",df.info(), "\n")
  print("Describe \n",df.describe(), "\n")
  print("Sum Null \n ",df.isnull().sum(), "\n")
  print("Sum Duplicated \n",df.duplicated().sum(), "\n")
  print("Colums \n",df.columns, "\n")
  print("Types \n",df.dtypes, "\n")


Head : 
    PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN     

In [415]:
def pre_process(df, train = True):
  df.drop(['PassengerId'], axis=1, inplace=True)
  df.drop(['Name'], axis=1, inplace=True)
  df.drop(['Ticket'], axis=1, inplace=True)
  for i in range (len(df.Embarked)):
    if type(df.Embarked[i]) == float:
      df.drop([i], axis=0, inplace=True)
  for i in df.Cabin.unique():
    if type(i) == float:
      df.replace(i, 0, inplace=True)
    else:
      df.replace(i, 1, inplace=True)
  for i in df.Age.unique():
    if type(i) == float:
      df.replace(i, df.Age.mean(), inplace=True)
    else:
      pass
  lb = LabelBinarizer()
  df['Sex'] = lb.fit_transform(df['Sex'])
  OH = OneHotEncoder(sparse=False)
  OH_df = pd.DataFrame(OH.fit_transform(df[['Embarked']]))
  OH_df.columns = OH.get_feature_names_out(['Embarked'])
  df = df.join(OH_df)

  OH = OneHotEncoder(sparse=False)
  OH_df = pd.DataFrame(OH.fit_transform(df[["Pclass"]]))
  OH_df.columns = OH.get_feature_names_out(['Pclass'])
  df = df.join(OH_df)

  df.drop(['Embarked', 'Pclass'], axis=1, inplace=True)


  sc = StandardScaler()
  df[['Age', 'Fare', 'SibSp','Parch']] = sc.fit_transform(df[['Age', 'Fare', 'SibSp','Parch']])

  df = df.dropna(subset=["Embarked_C"])
  df = df.dropna(subset=["Embarked_Q"])
  df = df.dropna(subset=["Embarked_S"])
  if train:
    survived = df['Survived']
    df.drop(['Survived'], axis=1, inplace=True)
  else:
    survived = None
  return df, survived


In [416]:
df, survived = pre_process(df)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [417]:
print(df.Cabin.value_counts())

Cabin
0    686
1    201
Name: count, dtype: int64


In [418]:
df["Sex"].value_counts()

Sex
1    575
0    312
Name: count, dtype: int64

In [419]:
print("Sum Null \n ",df.isnull().sum(), "\n")

Sum Null 
  Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin         0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Pclass_1      0
Pclass_2      0
Pclass_3      0
dtype: int64 



In [420]:
df.describe()

,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000
mean,0.648253,-0.000676,0.001071,0.001070,0.000600,0.226607,0.189402,0.086809,0.723788,0.240135,0.206313,0.553551
std,0.477785,1.001557,1.001437,1.001437,1.001555,0.418872,0.392049,0.281715,0.447375,0.427406,0.404886,0.497404
min,0.000000,-1.352516,-0.475199,-0.474326,-0.646204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-1.010689,-0.475199,-0.474326,-0.486944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.014793,-0.475199,-0.474326,-0.355197,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1.000000,0.641476,0.431350,-0.474326,-0.022080,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,3.205180,6.777195,6.967010,9.668551,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Train Model

In [421]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, survived, test_size=0.2)

In [422]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [423]:
model.score(X_train, y_train)

0.33554926378277494

In [424]:
model.score(X_test, y_test)

0.5095524383815482

In [425]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [426]:
model.score(X_train, y_train)

0.8236953455571228

In [427]:
model.score(X_test, y_test)

0.8370786516853933

In [428]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [429]:
model.score(X_train, y_train)

0.995768688293371

In [430]:
model.score(X_test, y_test)

0.8314606741573034

In [431]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [432]:
model.score(X_train, y_train)

0.995768688293371

In [433]:
model.score(X_test, y_test)

0.8539325842696629

In [435]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)

SVC()

In [436]:
model.score(X_train, y_train)

0.8194640338504936

In [437]:
model.score(X_test, y_test)

0.8707865168539326

In [438]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [439]:
model.score(X_train, y_train)

0.7616361071932299

In [440]:
model.score(X_test, y_test)

0.848314606741573

In [441]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [442]:
model.score(X_train, y_train)

0.8504936530324401

In [443]:
model.score(X_test, y_test)

0.8539325842696629